### energy

1. solar arrays
2. heat pumps
3. EV charges
4. storage tanks
6. propane tanks

In [2]:
import pandas as pd
import geopandas as gpd


In [133]:
data_dir = './'
a_solar = gpd.read_file(data_dir+'ArlingtonMA.geojson',dtypes={'tokenId':str})
cols = ['tokenId','image','dates','entity','financials','watts']
df = pd.DataFrame({
    'loc_id':[x['location'] for x in a_solar.governance],
    'pid':[x['parcel'] for x in a_solar.governance],
    'utility':['' if type(x)==float else x['utility'] if 'utility' in x else '' for x in a_solar.amps] ,
    'meter_mfgr':['' if type(x)==float else x['meter_mfgr'] if 'meter_mfgr' in x else '' for x in a_solar.amps] ,
})
for col in cols:
    df[col] = a_solar[col]


##construct image on client from lat/lon coords
#df.image[0]

int_value_pairs=pd.DataFrame()
for col in ['utility','meter_mfgr']:
    a = list(df[col].unique())
    xref = dict(zip(a,range(len(a))))
    tmp = pd.DataFrame({'key':xref.values(),'item':col,'value':xref.keys()})
    int_value_pairs = pd.concat([int_value_pairs,tmp])
    df[col]=df[col].replace(xref)

In [135]:
table_create_query = \
"""
        DROP TABLE IF EXISTS property.solar;
        CREATE TABLE property.solar (
            "loc_id" VARCHAR(16),
            "pid" VARCHAR(17),
            "utility" SMALLINT,
            "meter_mfgr" SMALLINT,
            "tokenId" CHAR(19), --DECIMAL(19,0),           
            "image" VARCHAR(125),
            "dates" JSONB ,
            "entity" JSONB ,
            "financials" JSONB ,
            "watts" JSONB 
        );
        
    CREATE INDEX solar_idx 
        ON property.solar(pid);
    CREATE INDEX solar_loc_id_idx 
        ON property.solar(loc_id);
    """
cnx.execute(table_create_query)


In [147]:
int_value_pairs.to_sql(
    'int_value_pairs',schema='common',con=cnx,
    if_exists='append',index=False
)

24

In [136]:
from sqlalchemy import dialects

df.to_sql(
    'solar',schema='property',con=cnx,
    if_exists='append',index=False,
    dtype={
        'dates': dialects.postgresql.JSONB,
        'entity': dialects.postgresql.JSONB,
        'financials': dialects.postgresql.JSONB,
        'watts': dialects.postgresql.JSONB,
    }
)

120